In [ ]:
###중요: LoRA 구현 부분
# -----------------------------------------------------------------------------
# 2. LoRA 클래스 및 함수 정의
# -----------------------------------------------------------------------------
class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        """
        LoRA(Low-Rank Adaptation) 레이어 초기화
        """
        super().__init__()
        self.A = nn.Parameter(torch.empty(in_dim, rank))
        # 입력을 저차원 공간(rank)으로 투영하는 역할.
        nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
        # kaiming_uniform_ 초기화로 학습 안정성을 확보. 그래서 앞에 empty로 하는듯?
        
        self.B = nn.Parameter(torch.zeros(rank, out_dim))
        # 저차원 표현을 다시 원래 출력 차원으로 확장하는 역할.
        # 학습 초기에 원래 모델의 출력에 영향을 주지 않도록 설계.
        
        self.alpha = alpha
        self.rank = rank
    def forward(self, x):
        x = (self.alpha / self.rank) * (x @ self.A @ self.B)
        # LoRA 논문에서 제안된 안정화 기법입니다. scaling factor = self.alpha / self.rank
        return x

In [ ]:
class LinearWithLoRA(nn.Module):
    """
    기존의 Linear 레이어를 감싸서 LoRA 어댑터를 추가한 클래스
    """
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        # 기존 Linear와 LoRA 출력 합산 x @ W + (self.alpha / self.rank) * (x @ A @ B) 가 되도록
        return self.linear(x) + self.lora(x)


In [ ]:
def replace_linear_with_lora(model, rank, alpha):
    """
    모델 내의 모든 Linear 레이어를 찾아 LoRA가 적용된 레이어로 교체
    """
    for name, module in model.named_children():
        if isinstance(module, torch.nn.Linear):
            # 기존 Linear 레이어를 LinearWithLoRA로 교체
            setattr(model, name, LinearWithLoRA(module, rank, alpha))
        else:
            # 재귀 호출
            replace_linear_with_lora(module, rank, alpha)